In [7]:
!pip install datatable

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for datatable from https://files.pythonhosted.org/packages/bb/c5/987fcb116df777d2573c8918b7d7bb391405fa0ca3ed209238bf447b7aac/datatable-1.1.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB 7.0 MB/s eta 0:00:01
   -------- ------------------------------- 1.0/4.7 MB 13.3 MB/s eta 0:00:01
   ----------------- ---------------------- 2.1/4.7 MB 16.6 MB/s eta 0:00:01
   ----------------- ---------------------- 2.1/4.7 MB 16.6 MB/s eta 0:00:01
   -------------------------- ------------- 3.1/4.7 MB 16.8 MB/s eta 0:00:01
   ----------------------------------- ---- 4.2/4.7 MB 15.8 MB/s eta 0:00:01
   ----------------------------------- ---- 4.2/4.7 MB 15.8 MB/s eta 0:00:01
   ---------------------------------------- 4.7/4.7 MB 13.7 MB/s eta 0:00:00


In [1]:
# import json
# import csv

# def json_to_csv(input_file, output_file):
#     with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
#         fieldnames = ['hotel_url', 'author', 'date', 'rating', 'title', 'text', 'property_dict']
#         csv_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
#         csv_writer.writeheader()

#         for line in infile:
#             data = json.loads(line)
#             csv_writer.writerow(data)

# # Example usage:
# input_file = '../hotelRec/HotelRec.txt'
# output_file = 'hotel_reviews.csv'
# json_to_csv(input_file, output_file)

In [2]:
num_rows = len(df.index)
print(num_rows)

50264531


In [2]:
import dask.dataframe as dd

df = dd.read_csv('../data/hotel_reviews.csv')
# Perform computations on the Dask DataFrame
print(df.head())

                                           hotel_url        author  \
0  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...  violettaf340   
1  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...     Lagaiuzza   
2  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...    ashleyn763   
3  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...   DavideMauro   
4  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...      Alemma11   

                  date  rating  \
0  2019-01-01T00:00:00     5.0   
1  2016-01-01T00:00:00     5.0   
2  2014-10-01T00:00:00     5.0   
3  2014-08-01T00:00:00     5.0   
4  2013-08-01T00:00:00     4.0   

                                              title  \
0                                      Xmas holiday   
1                                Baltic, what else?   
2                           Excellent in every way!   
3                The house of your family's holiday   
4  A paradise for children (and parents, of course)   

                                           

In [3]:
from typing import Dict, Tuple
import pandas as pd

def create_label_encoders(df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[str, int]]:
    """
    Create label encoders (mapping dictionaries) for hotel_url and author columns
    
    Args:
        df (pd.DataFrame): Input dataframe containing hotel_url and author columns
        
    Returns:
        Tuple[Dict[str, int], Dict[str, int]]: Hotel URL and author mapping dictionaries
    """
    # Create mapping dictionaries
    hotel_mapping = {url: idx for idx, url in enumerate(df['hotel_url'].unique())}
    author_mapping = {author: idx for idx, author in enumerate(df['author'].unique())}
    
    return hotel_mapping, author_mapping

def encode_hotel_author(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode hotel_url and author columns to integer values
    
    Args:
        df (pd.DataFrame): Input dataframe containing hotel_url and author columns
        
    Returns:
        pd.DataFrame: DataFrame with encoded columns added
    """
    # Create a copy to avoid modifying the original dataframe
    df_encoded = df.copy()
    
    # Create mapping dictionaries
    hotel_mapping, author_mapping = create_label_encoders(df)
    
    # Add encoded columns
    df_encoded['hotel_id'] = df_encoded['hotel_url'].map(hotel_mapping)
    df_encoded['author_id'] = df_encoded['author'].map(author_mapping)
    
    # Print some statistics
    print(f"Number of unique hotels: {len(hotel_mapping)}")
    print(f"Number of unique authors: {len(author_mapping)}")
    
    return df_encoded, hotel_mapping, author_mapping

In [4]:
# Encode the data
df_encoded, hotel_mapping, author_mapping = encode_hotel_author(df)

# Now df_encoded contains new columns 'hotel_id' and 'author_id'
# The original columns are preserved

# Example usage:
print("\nSample of encoded data:")
print(df_encoded[['hotel_url', 'hotel_id', 'author', 'author_id']].head())




Number of unique hotels: 365057
Number of unique authors: 21891404

Sample of encoded data:
                                           hotel_url  hotel_id        author  \
0  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...         0  violettaf340   
1  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...         0     Lagaiuzza   
2  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...         0    ashleyn763   
3  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...         0   DavideMauro   
4  Hotel_Review-g194775-d1121769-Reviews-Hotel_Ba...         0      Alemma11   

   author_id  
0          0  
1          1  
2          2  
3          3  
4          4  


In [8]:
# Remove hotel_url and author columns
df_encoded = df_encoded.drop(['hotel_url', 'author'], axis=1)
# keeping rows with rating value, removing empty rating rows
df_encoded = df_encoded[df_encoded['rating'].notnull()]

# # Step 1: Get value counts for authors
# print("Counting reviews per author...")
# author_counts = df_encoded['author_id'].value_counts().compute()

# # Step 2: Get list of authors with more than 20 reviews
# print("Getting list of frequent authors...")
# frequent_authors = list(author_counts[author_counts >= 20].index)
# print(f"Found {len(frequent_authors)} authors with 20+ reviews")

# # Step 3: Filter and save in chunks
# print("Filtering and saving data...")
# df_frequent = df_encoded[df_encoded['author_id'].isin(frequent_authors)]

# # Save to CSV in chunks
# df_frequent.to_csv('../data/hotel_reviews_frequent_reviewers_*.csv', 
#                   single_file=True,  # Combines all parts into a single file
#                   compute=True,      # Actually performs the computation
#                   encoding='utf-8',
#                   index=False)

# print("Done! Checking file size...")

# # Check results
# import os
# import glob

# # Get the saved file(s)
# saved_file = glob.glob('../data/hotel_reviews_frequent_reviewers_*.csv')[0]
# file_size = os.path.getsize(saved_file) / (1024 * 1024)  # Convert to MB
# print(f"File saved as: {saved_file}")
# print(f"File size: {file_size:.2f} MB")

ValueError: Metadata inference failed in `drop_by_shallow_copy`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KeyError("['hotel_url', 'author'] not found in axis")

Traceback:
---------
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\utils.py", line 193, in raise_on_meta_error
    yield
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\core.py", line 6876, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\utils.py", line 774, in drop_by_shallow_copy
    df2.drop(columns=columns, inplace=True, errors=errors)
  File "C:\ProgramData\anacon

In [12]:
# optimized version to keep only authors with 20+ reviews
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Enable progress bar
pbar = ProgressBar()
pbar.register()

# Optimize partitions based on memory
print("Computing npartitions...")
npartitions = 25
df_encoded = df_encoded.repartition(npartitions=npartitions)

# Compute author counts using optimized aggregation
print("Computing author frequencies...")
author_counts = df_encoded.author_id.value_counts(split_every=8).compute()
print("Computing frequent authors...")
frequent_authors = set(author_counts[author_counts >= 20].index)
print(f"Found {len(frequent_authors)} frequent authors")

# Filter with optimized settings
print("Filtering and saving data...")
df_frequent = df_encoded[df_encoded.author_id.isin(frequent_authors)]

# Save with optimized settings
df_frequent.to_csv('../data/hotel_reviews_frequent_reviewers_*.csv',
                   single_file=True,
                   compute=True,
                   encoding='utf-8',
                   index=False,
                   compression='gzip',
                   blocksize='64MB')

print("Processing complete!")

Computing npartitions...
Computing author frequencies...
[########################################] | 100% Completed | 3hr 31m
[########################################] | 100% Completed | 3hr 31m
[########################################] | 100% Completed | 3hr 31m
Computing frequent authors...
Found 189992 frequent authors
Filtering and saving data...


IndexError: list index out of range

In [13]:
df_frequent.head()

[########################################] | 100% Completed | 486.53 s
[########################################] | 100% Completed | 486.63 s
[########################################] | 100% Completed | 486.73 s


,date,rating,title,text,property_dict,hotel_id,author_id
8,2017-03-01T00:00:00,5.0,Natural Luxury,"The property is surrounded by trees, which are...","{'service': 5.0, 'sleep quality': 5.0, 'value'...",1,8
12,2016-01-01T00:00:00,5.0,"Loved out stay, a lot of character","Well, this hotel stands out from the crowd, so...","{'rooms': 5.0, 'service': 5.0, 'cleanliness': ...",1,12
14,2015-07-01T00:00:00,5.0,"Really nice staff, yummy breakfast and the mon...",The pousada is like 30 minute walk to the cent...,"{'rooms': 5.0, 'service': 5.0, 'value': 5.0}",1,8
24,2017-05-01T00:00:00,4.0,Gorgeous agriturismo with beautiful grounds an...,We made a brief stop to visit the 12th century...,{'location': 4.0},2,23
25,2017-01-01T00:00:00,5.0,Really enjoyed the experience,Recently spent 2 nights here and used it as a ...,"{'rooms': 5.0, 'service': 5.0, 'sleep quality'...",2,24


In [20]:
import pandas as pd

# Now df_frequent contains only entries where the author has more than 20 reviews
# Export the filtered DataFrame to a CSV file
df_frequent.to_csv('../data/filtered_reviews.csv', index=False)

print("done")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\cyclo\AppData\Local\Temp\ipykernel_14516\1072007154.py", line 5, in <module>
    df_frequent.to_csv('../data/filtered_reviews.csv', index=False)
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\core.py", line 1840, in to_csv
    return to_csv(self, filename, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py", line 939, in to_csv
    dfs = df.to_delayed()
          ^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\core.py", line 1903, in to_delayed
    graph = self.__dask_optimize__(graph, self.__dask_keys__())
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\dask\dataframe\optimize.p

In [17]:
chunk_size = 50000

# Export the filtered DataFrame to CSV in chunks
df_frequent.to_csv('filtered_reviews.csv', index=False, chunksize=chunk_size)

[########################################] | 100% Completed | 3hr 44m
[########################################] | 100% Completed | 3hr 44m
[########################################] | 100% Completed | 3hr 44m


['C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\00.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\01.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\02.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\03.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\04.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\05.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\06.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\07.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\08.part',
 'C:\\Users\\cyclo\\OneDrive\\Documents\\GitHub\\SmartStay\\src\\filtered_reviews.csv\\09.part',
 'C:\\Users\\cyclo\\OneDrive\\

In [19]:
import os

# Path to the folder containing the .part files
folder_path = './filtered_reviews.csv'  # folder path

# Output CSV file
output_file = 'combined_filtered_reviews.csv'

# Open the output file in write mode with UTF-8 encoding
with open(output_file, 'w', encoding='utf-8') as outfile:
    first_file = True  # To track the first file for including headers

    # Iterate through all .part files in the folder (sorted if needed)
    for part_file in sorted(os.listdir(folder_path)):
        if part_file.endswith('.part'):  # Only process .part files
            with open(os.path.join(folder_path, part_file), 'r', encoding='utf-8') as infile:
                if first_file:
                    # Write the header from the first file
                    outfile.write(infile.read())
                    first_file = False
                else:
                    # Skip the header for subsequent files
                    next(infile)  # Skip the first line
                    outfile.write(infile.read())


In [22]:
df_new = dd.read_csv('combined_filtered_reviews.csv')
# Perform computations on the Dask DataFrame
print(df_new.head(15))

[########################################] | 100% Completed | 922.12 ms
[########################################] | 100% Completed | 1.02 s
[########################################] | 100% Completed | 1.11 s
                   date  rating  \
0   2017-03-01T00:00:00     5.0   
1   2016-01-01T00:00:00     5.0   
2   2015-07-01T00:00:00     5.0   
3   2017-05-01T00:00:00     4.0   
4   2017-01-01T00:00:00     5.0   
5   2016-10-01T00:00:00     5.0   
6   2016-10-01T00:00:00     4.0   
7   2016-10-01T00:00:00     4.0   
8   2016-05-01T00:00:00     5.0   
9   2016-05-01T00:00:00     4.0   
10  2015-09-01T00:00:00     5.0   
11  2015-06-01T00:00:00     5.0   
12  2015-10-01T00:00:00     5.0   
13  2015-09-01T00:00:00     5.0   
14  2019-02-01T00:00:00     3.0   

                                                title  \
0                                      Natural Luxury   
1                  Loved out stay, a lot of character   
2   Really nice staff, yummy breakfast and the mon...   
3